In [1]:
import os
import anndata
import cooler
import pandas as pd
from joblib import Parallel, delayed

In [2]:
folder_path = '/home/micl/workspace/lmh_data/Lee2019/Human_single_cell_10kb_cool'

parallel = Parallel(n_jobs=48, backend='loky', verbose=1)

def load_coolers(folder_path):
    def load_cooler(folder_path, file_name):
        c = cooler.Cooler(os.path.join(folder_path, file_name))
        contact = c.pixels(join=True)[:]
        contact = contact[contact['start1']!=contact['start2']]
        binsize, chromsizes = c.binsize, c.chromsizes

        _1 = contact.groupby(['chrom1', 'start1'])['count'].sum()
        _2 = contact.groupby(['chrom2', 'start2'])['count'].sum()
        _1.index.names = _2.index.names = ['chrom', 'start']
        _1, _2 = _1[_1!=0], _2[_2!=0]
        info = pd.concat([_1, _2], axis=1).fillna(0).sum(axis=1).sort_index()
        
        _indexs = set([(chrom, int(i * binsize))
                   for chrom in chromsizes.keys()
                   for i in range(int(chromsizes[chrom]/binsize)+1)])
        _indexs -= set(info.index)
        info = pd.concat([info, pd.Series([0]*len(_indexs), index=list(_indexs))]).sort_index()

        return info.to_frame().astype('float16').rename(columns={0:file_name})

    joblist = []
    for root, dirs, files in os.walk(folder_path, topdown=False):
        for file_name in files:
            joblist.append(delayed(load_cooler)(folder_path, file_name))

    infos = parallel(joblist)
    infos = pd.concat(infos, axis=1).fillna(0).sort_index()
    return infos

infos = load_coolers(folder_path)
infos

[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:   19.3s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:   53.5s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:  1.7min
[Parallel(n_jobs=48)]: Done 1154 tasks      | elapsed:  2.7min
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:  4.0min
[Parallel(n_jobs=48)]: Done 2354 tasks      | elapsed:  5.5min
[Parallel(n_jobs=48)]: Done 3104 tasks      | elapsed:  7.2min
[Parallel(n_jobs=48)]: Done 3954 tasks      | elapsed:  9.2min
[Parallel(n_jobs=48)]: Done 4238 out of 4238 | elapsed:  9.8min finished


190315_29yr_6_E11_AD008_ODC_10kb_contacts.cool  \
chrom start                                                      
chr1  0                                                    0.0   
      10000                                                1.0   
      20000                                                0.0   
      30000                                                0.0   
      40000                                                1.0   
...                                                        ...   
chrY  59330000                                             0.0   
      59340000                                             0.0   
      59350000                                             0.0   
      59360000                                             0.0   
      59370000                                             0.0   

                181218_21yr_2_B11_AD002_Vip_10kb_contacts.cool  \
chrom start                                                      
chr1  0                                                    0.0   
      10000                                                0.0   
      20000                                                0.0   
      30000                                                0.0   
      40000                                                0.0   
...                                                        ...   
chrY  59330000                                             0.0   
      59340000                                             0.0   
      59350000                                             0.0   
      59360000                                             0.0   
      59370000                                             0.0   

                190315_29yr_4_A3_AD006_L5_10kb_contacts.cool  \
chrom start                                                    
chr1  0                                                  0.0   
      10000                                              0.0   
      20000                                              0.0   
      30000                                              0.0   
      40000                                              0.0   
...                                                      ...   
chrY  59330000                                           0.0   
      59340000                                           0.0   
      59350000                                           0.0   
      59360000                                           0.0   
      59370000                                           0.0   

                190315_21yr_6_F11_AD012_L23_10kb_contacts.cool  \
chrom start                                                      
chr1  0                                                    0.0   
      10000                                                0.0   
      20000                                                0.0   
      30000                                                0.0   
      40000                                                0.0   
...                                                        ...   
chrY  59330000                                             0.0   
      59340000                                             0.0   
      59350000                                             0.0   
      59360000                                             0.0   
      59370000                                             0.0   

                181218_21yr_2_G12_AD010_MP_10kb_contacts.cool  \
chrom start                                                     
chr1  0                                                   0.0   
      10000                                               0.0   
      20000                                               0.0   
      30000                                               0.0   
      40000                                               0.0   
...                                                       ...   
chrY  59330000                                            0.0   
      59340000       

In [3]:
obs = pd.DataFrame(infos.T.index, columns=['cells'])
obs.insert(obs.shape[1] - 1, 'domain', 'scHiC')
obs = obs.set_index('cells')
var = infos.reset_index()[['chrom', 'start']].set_index(infos.index.map('{0[0]}_{0[1]}'.format))

infos.index = infos.index.map('{0[0]}_{0[1]}'.format)
infos = anndata.AnnData(X=infos.T, obs=obs, var=var)

/tmp/ipykernel_3505395/2898016580.py:7: FutureWarning: X.dtype being converted to np.float32 from float16. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  infos = anndata.AnnData(X=infos.T, obs=obs, var=var)


In [4]:
infos.write("/home/micl/workspace/lmh_data/Lee2019/scHiC.h5ad", compression="gzip")